In [20]:
import pdfplumber
import re
from datetime import timedelta, date, datetime
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
import plotly.graph_objects as go


In [4]:
def file_path_fcn(file_path=None):
    if file_path is None:
        file_path = 'REPORTES/LUPITA.pdf'
    return file_path

file_path ='REPORTES/BOYER.pdf'

# file_path = file_path_fcn(file_path=file_path)
file_path = file_path_fcn()
file_path

'REPORTES/LUPITA.pdf'

In [5]:
def pdf_a_texto(file_path):
    '''Conversion del pdf a texto'''
    with pdfplumber.open(file_path) as pdf:
        text = ''
        for page in pdf.pages:
            text += page.extract_text()
    return text

texto = pdf_a_texto(file_path)

In [6]:
def format_spanish_date(my_date, date_type="format_string"):
    days = ["Lunes", "Martes", "Miércoles", "Jueves",
            "Viernes", "Sábado", "Domingo"]
    months = ["enero", "febrero", "marzo", "abril", "mayo", "junio",
              "julio", "agosto", "septiembre", "octubre", "noviembre", "diciembre"]

    for fmt in ("%d/%m/%Y", "%d-%m-%Y", "%Y-%m-%d", "%Y/%m/%d"):
        # here we will check if date_type is "format_string" or "date"
        if date_type == "format_string":
            try:
                date_obj = datetime.strptime(my_date, fmt)
                break
            except ValueError:
                continue
        elif date_type == "format_date":
            try:
                date_obj = datetime.strptime(my_date, fmt)
                break
            except ValueError:
                continue
        else:
            raise ValueError("Invalid date_type. It should be 'format_string' or 'date'.")

    day_name = days[date_obj.weekday()]
    month_name = months[date_obj.month - 1]
    day_number = date_obj.day
    year = date_obj.year

    if date_type == "format_string":
        return f"{day_name}, {day_number} de {month_name} de {year}"
    elif date_type == "format_date":
        return date_obj.date()


# Example usage:
date_str = "17/02/1992"
# date_str = '2024-09-19'
# date_str = '1964/10/13'
formatted_date = format_spanish_date(date_str, date_type="format_date")
# formatted_date = format_spanish_date(date_str, date_type="format_string")
print(formatted_date)

1992-02-17


In [7]:
def convert_double_currency(value):
    # We will receive a float value and return a string with $ and 2 decimals
    return "${:,.2f}".format(value)

# Example usage:
value = 1234.56
formatted_value = convert_double_currency(value)
print(formatted_value)

$1,234.56


In [8]:
class DatosGenerales:
    def __init__(self, texto):
        self.texto = texto

    def nombrefcn(self):
    # Extract the substring between the found positions, adjust indices for correct slicing
        InicioStr = re.search(r"20\d{2}\s{1}", self.texto)
        FinalStr = "DD MM YYYY"
        # Find the starting and ending positions of the target substrings
        Inicio = InicioStr.span()[0]
        Final = texto.find(FinalStr)
        # Extract the substring between the found positions, adjust indices for correct slicing
        if Inicio != -1 and Final != -1:
            # + 5 since InicioStr length = length(20) + 2 + 1 = 5
            Nombre = self.texto[Inicio + 5: Final].strip()
        else:
            Nombre = ""
        print(f"NOMBRE: {Nombre}")
        self.Nombre = Nombre

    def nssfcn(self):
        InicioStr = "NSS: "
        FinalStr = "CURP:"

        Inicio = self.texto.find(InicioStr)
        Final = self.texto.find(FinalStr)

        if Inicio != -1 and Final != -1:
            nss = self.texto[Inicio + 5: Final - 28].strip()
        else:
            nss = ""
        print(f"NSS: {nss}")
        self.nss = nss

    def curpfcn(self):
        InicioStr = "CURP:"

        Inicio = self.texto.find(InicioStr)

        if Inicio != -1:
            curp = self.texto[Inicio +len(InicioStr): Inicio + 25].strip()
        else:
            curp = ''
        print(f"CURP: {curp}")
        self.curp = curp

    def fecha_nacimiento_fcn(self):
        YEAR_BIRTH = "19" + self.curp[4:6]
        MONTH_BIRTH = self.curp[6:8]
        DAY_BIRTH = self.curp[8:10]
        FECHA_NACIMIENTO = DAY_BIRTH + '/' + MONTH_BIRTH + '/' + YEAR_BIRTH
        # Convert string to datetime object
        FECHA_NACIMIENTO_date = format_spanish_date(FECHA_NACIMIENTO, date_type="format_date")
        # Convert string to spanish date string
        FECHA_NACIMIENTO_string = format_spanish_date(FECHA_NACIMIENTO, date_type="format_string")
        print(f"FECHA DE NACIMIENTO: {FECHA_NACIMIENTO}")
        self.fecha_nacimiento_string = FECHA_NACIMIENTO_string
        self.fecha_nacimiento_date = FECHA_NACIMIENTO_date

    def ano_inscripcionIMSS_fcn(self):
        TerminacionYear = self.nss[2:4]
        if int(TerminacionYear) < 50:
            AnoInicio = int('20'+ TerminacionYear)
        else:
            AnoInicio = int('19'+ TerminacionYear)
        print(f"AÑO DE INSCRIPCIÓN IMSS: {AnoInicio}")
        self.AnoInicio = AnoInicio

    def regimen_pertenecefcn(self):
        if self.AnoInicio > 1997:
            Regimen = 'Régimen 97'
        elif self.AnoInicio == 1997:
            Regimen = "Determinar el mes que inició, si es a partir de julio, entra en régimen 97"
        else:
            Regimen = 'Régimen 73'

        print(f"REGIMEN AL QUE PERTENECE: {Regimen}")
        self.Regimen = Regimen

    def semanas_cotizadas_descontadas_fcn(self):
        Iniciostr = re.search(r"\d{1,4}\s{1}\d{1,4}\s{1}\d{1,4}", texto)
        myrange = Iniciostr.span()
        risultati = self.texto[myrange[0]:myrange[1]]
        semanas_cotizadas = int(risultati.split()[0])
        semanas_descontadas = int(risultati.split()[1])
        semanas_reintegradas = int(risultati.split()[-1])
        semanas_totales = semanas_cotizadas - semanas_descontadas + semanas_reintegradas

        print(f"SEMANAS COTIZADAS: {semanas_cotizadas}")
        print(f"SEMANAS DESCONTADAS: {semanas_descontadas}")
        print(f"SEMANAS REINTEGRADAS: {semanas_reintegradas}")
        print(f"SEMANAS TOTALES: {semanas_totales}")

        self.semanas_cotizadas = semanas_cotizadas
        self.semanas_descontadas = semanas_descontadas
        self.semanas_reintegradas = semanas_reintegradas
        self.semanas_totales = semanas_totales

    def fecha_emision_reporte_fcn(self):
        texto_array = self.texto.split('\n')
        # Find the position of the string that ends with 'reporte' and select the next element that contains the date
        position_date_report = [i + 1 for i, s in enumerate(texto_array) if s.endswith('reporte')]
        out = texto_array[position_date_report[0]]
        out = out.replace(' ', '')
        self.fecha_emision_reporte_date = format_spanish_date(out, date_type="format_date")
        self.fecha_emision_reporte_string = format_spanish_date(out, date_type="format_string")

        print(f"FECHA DE EMISIÓN DEL REPORTE: {self.fecha_emision_reporte_date}, type: {type(self.fecha_emision_reporte_date)}")

    def tabla_datos(self):
        self.nombrefcn()
        self.nssfcn()
        self.curpfcn()
        self.fecha_nacimiento_fcn()
        self.ano_inscripcionIMSS_fcn()
        self.regimen_pertenecefcn()
        self.semanas_cotizadas_descontadas_fcn()
        self.fecha_emision_reporte_fcn()

        out = pd.DataFrame({
            'NOMBRE': [self.Nombre],
            'NSS': [self.nss],
            'CURP': [self.curp],
            'FECHA DE NACIMIENTO': [self.fecha_nacimiento_string],
            'AÑO DE INSCRIPCIÓN IMSS': [self.AnoInicio],
            'REGIMEN AL QUE PERTENECE': [self.Regimen],
            'SEMANAS COTIZADAS': [self.semanas_cotizadas],
            'SEMANAS DESCONTADAS': [self.semanas_descontadas],
            'SEMANAS REINTEGRADAS': [self.semanas_reintegradas],
            'SEMANAS TOTALES': [self.semanas_totales],
            'FECHA DE EMISIÓN DEL REPORTE': [self.fecha_emision_reporte_string]
        }
        )
        out = out.T
        out.columns = ['DATOS PERSONALES']
        return out

# Example usage
Usuario = DatosGenerales(texto)
df = Usuario.tabla_datos()
df

NOMBRE: SAN ROMAN MOSCOSO MARIA GUADALUPE
NSS: 71896250306
CURP: SAMG620423MCSNSD01
FECHA DE NACIMIENTO: 23/04/1962
AÑO DE INSCRIPCIÓN IMSS: 1989
REGIMEN AL QUE PERTENECE: Régimen 73
SEMANAS COTIZADAS: 805
SEMANAS DESCONTADAS: 0
SEMANAS REINTEGRADAS: 0
SEMANAS TOTALES: 805
FECHA DE EMISIÓN DEL REPORTE: 2021-03-03, type: <class 'datetime.date'>


,DATOS PERSONALES
NOMBRE,SAN ROMAN MOSCOSO MARIA GUADALUPE
NSS,71896250306
CURP,SAMG620423MCSNSD01
FECHA DE NACIMIENTO,"Lunes, 23 de abril de 1962"
AÑO DE INSCRIPCIÓN IMSS,1989
REGIMEN AL QUE PERTENECE,Régimen 73
SEMANAS COTIZADAS,805
SEMANAS DESCONTADAS,0
SEMANAS REINTEGRADAS,0
SEMANAS TOTALES,805


In [9]:
class FechasGenerales:
    def __init__(self, texto, FechaEmisionReporte):
        self.texto = texto
        self.FechaEmisionReporte = FechaEmisionReporte

    def tabla_fechas_generales(self):
        texto = self.texto
        FechaEmisionReporte = self.FechaEmisionReporte
        # Define the search string
        bajas_string = 'Fecha de baja'
        value_added = 12 # Assuming 12 characters for the date
        # Find the indices of the search string
        bajas = [m.start() for m in re.finditer(bajas_string, texto)]
        # Get the index of the first occurrence
        ultima_baja = bajas[0]
        # Calculate the end index
        ultima_final = ultima_baja + len(bajas_string) + value_added
        # Extract the desired substring
        sigue_cotizando = "vigente" in texto.lower()  #Check if sigue_cotizando is contained in the text
        # Initialize an array to store dates
        fecha_bajas = np.zeros(len(bajas), dtype=object)
        fecha_bajas_string = np.zeros(len(bajas), dtype=object)

        for idx, baja in enumerate(bajas):
            if idx == 0:
                if not sigue_cotizando:
                    fecha_unstructured = texto[ultima_baja + len(bajas_string):ultima_final].strip()
                    fechas_ultima_baja = format_spanish_date(fecha_unstructured, date_type="format_date")
                    fecha_bajas[idx] = fechas_ultima_baja
                    fecha_bajas_string[idx] = format_spanish_date(fecha_unstructured, date_type="format_string")
                else:
                    fecha_bajas[idx] = FechaEmisionReporte
                    fecha_bajas_string[idx] = format_spanish_date(fecha_bajas[idx], date_type="format_string")
                    fechas_ultima_baja = fecha_bajas[idx]

                if not isinstance(fechas_ultima_baja, date):
                    fechas_ultima_baja = fechas_ultima_baja.date()
            else:
                start_idx = baja + len(bajas_string)
                end_idx = start_idx + value_added  # Assuming 12 characters for the date
                fecha = texto[start_idx:end_idx].strip()
                fecha_bajas[idx] = format_spanish_date(fecha,  date_type="format_date")
                fecha_bajas_string[idx] = format_spanish_date(fecha, date_type="format_string")

        altas_string = 'Fecha de alta'
        altas = [m.start() for m in re.finditer(altas_string, texto)]

        fecha_altas = np.zeros(len(altas), dtype=object)
        fecha_altas_string = np.zeros(len(altas), dtype=object)
        dias_transcurridos_cotizados = np.zeros(len(altas), dtype=object)
        semanas_transcurridas_cotizadas = np.zeros(len(altas), dtype=object)

        for idx, alta in enumerate(altas):
            start_idx = alta + len(altas_string)
            end_idx = start_idx + value_added

            fecha = texto[start_idx:end_idx].strip()
            fecha_altas[idx] = format_spanish_date(fecha, date_type="format_date")
            fecha_altas_string[idx] = format_spanish_date(fecha, date_type="format_string")

            dias_transcurridos_cotizados[idx] = (fecha_bajas[idx] - fecha_altas[idx]).days
            semanas_transcurridas_cotizadas[idx] = dias_transcurridos_cotizados[idx] // 7

            inicio_patron_str = 'Nombre del patrón'
            final_patron_str = 'Registro Patronal'
            final_patron_str_entidad = 'Entidad federativa'
            inicio_patron = [m.start() for m in re.finditer(inicio_patron_str, texto)]
            final_patron = [m.start() for m in re.finditer(final_patron_str, texto)]

            if len(final_patron) != len(inicio_patron):
                missing_registro_patronal = [m.start() for m in re.finditer(final_patron_str_entidad, texto)]
                final_patron.append(missing_registro_patronal[-1])

            # using list comprehension
            patrones = [texto[inicio + len(inicio_patron_str):final_patron[idx] - 1].strip()
                        for idx, inicio in enumerate(inicio_patron)]

            inicio_ef_str = 'Entidad federativa'
            final_ef_str = 'Fecha de alta'
            inicio_ef = [m.start() for m in re.finditer(inicio_ef_str, texto)]
            final_ef = [m.start() for m in re.finditer(final_ef_str, texto)]

            entidad_federativa = []

            for idx, inicio in enumerate(inicio_ef):
                start_idx = inicio + len(inicio_ef_str)
                end_idx = final_ef[idx] - 1
                entidad_federativa.append(texto[start_idx:end_idx].strip())

            # Alternatively, using list comprehension
            entidad_federativa = [texto[inicio + len(inicio_ef_str):final_ef[idx] - 1].strip()
                                for idx, inicio in enumerate(inicio_ef)]

            FechasGenerales_num = pd.DataFrame({
            'Fecha de alta': fecha_altas,
            'Fecha de baja': fecha_bajas,
            'Dias transcurridos cotizados': dias_transcurridos_cotizados,
            'Semanas transcurridas cotizadas': semanas_transcurridas_cotizadas,
            'Patrones': patrones,
            'Entidad federativa': entidad_federativa
        })
            FechasGenerales_string = pd.DataFrame({
            'Fecha de alta': fecha_altas_string,
            'Fecha de baja': fecha_bajas_string,
            'Dias transcurridos cotizados': dias_transcurridos_cotizados,
            'Semanas transcurridas cotizadas': semanas_transcurridas_cotizadas,
            'Patrones': patrones,
            'Entidad federativa': entidad_federativa
        })
        return FechasGenerales_num, FechasGenerales_string, sigue_cotizando, fechas_ultima_baja

fechas_generales = FechasGenerales(texto=texto, FechaEmisionReporte=Usuario.fecha_emision_reporte_string)
fechasGeneralesNumerico, fechasGeneralesDate, sigueCotizando, fechasUltimaBaja = fechas_generales.tabla_fechas_generales()
print("Tabla Numérica")
display(fechasGeneralesNumerico)
print("Tabla Date")
display(fechasGeneralesDate)
print("Sigue Cotizando", sigueCotizando)
print(fechasUltimaBaja)

Tabla Numérica


,Fecha de alta,Fecha de baja,Dias transcurridos cotizados,Semanas transcurridas cotizadas,Patrones,Entidad federativa
0,2010-08-01,2020-09-30,3713,530,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,1999-10-16,2000-05-15,212,30,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,1994-10-01,1996-05-31,608,86,COLEGIO OFELIA NARVAEZ RINCON,CHIAPAS
3,1991-10-01,1992-09-10,345,49,COLEGIO OFELIA NARVAEZ RINCON,CHIAPAS
4,1989-09-01,1991-09-23,752,107,CTRO ESTUDIOS LA SALLE,CHIAPAS


Tabla Date


,Fecha de alta,Fecha de baja,Dias transcurridos cotizados,Semanas transcurridas cotizadas,Patrones,Entidad federativa
0,"Domingo, 1 de agosto de 2010","Miércoles, 30 de septiembre de 2020",3713,530,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,"Sábado, 16 de octubre de 1999","Lunes, 15 de mayo de 2000",212,30,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,"Sábado, 1 de octubre de 1994","Viernes, 31 de mayo de 1996",608,86,COLEGIO OFELIA NARVAEZ RINCON,CHIAPAS
3,"Martes, 1 de octubre de 1991","Jueves, 10 de septiembre de 1992",345,49,COLEGIO OFELIA NARVAEZ RINCON,CHIAPAS
4,"Viernes, 1 de septiembre de 1989","Lunes, 23 de septiembre de 1991",752,107,CTRO ESTUDIOS LA SALLE,CHIAPAS


Sigue Cotizando False
2020-09-30


# Historial Laboral Desglosado

In [10]:
def HistorialLaboralTabla_fcn(texto):
    # Define the search string
    BloqueInicio = 'Nombre del patrón'
    BloqueFinal = '* Valor del último salario base de cotización diario en pesos.'

    # Find occurrences of the start and end blocks
    Inicio = [m.start() for m in re.finditer(re.escape(BloqueInicio), texto)]
    Final = [m.start() for m in re.finditer(re.escape(BloqueFinal), texto)]
    HistoriaLaboral = []

    # Extract information between blocks
    for start, end in zip(Inicio, Final):
        substring = texto[start + len(BloqueInicio):end].strip()
        output = [line.strip() for line in substring.splitlines() if len(line.strip()) > 1]
        HistoriaLaboral.append(output)

    # Initialize an empty DataFrame for the final table
    target_strings = {'ALTA', 'REINGRESO', 'MODIFICACION', 'BAJA'}
    Movimiento = list()
    FechaMovimiento = list()
    Sueldo = list()
    Empleador = list()
    EntidadFederativa = list()
    for entry in HistoriaLaboral:
        for line in entry:
            # if any(string in line for string in target_strings):
            if any(string in line.split()[0] for string in target_strings):
                Empleador.append(entry[0])
                EntidadFederativa.append(entry[2].split()[2:])
                array = line.split()
                if array[0] == 'MODIFICACION':
                    Movimiento.append('MODIFICACION DE SALARIO')
                else:
                    Movimiento.append(array[0])
                FechaMovimiento.append(array[-3])
                Sueldo.append(array[-1])

    # Convert 'Sueldo' elements to float before formatting
    Sueldo_str = [convert_double_currency(float(valor)) for valor in Sueldo]
    HistoriaLaboralTable_str = pd.DataFrame({
        'Movimiento': Movimiento,
        'Fecha de Movimiento': FechaMovimiento,
        'Sueldo': Sueldo_str,
        'Empleador': Empleador,
        'Entidad Federativa': EntidadFederativa
    })

    HistoriaLaboralTable_str['Entidad Federativa'] = HistoriaLaboralTable_str['Entidad Federativa'].apply(lambda x: " ".join(x))

    HistoriaLaboralTable_num = HistoriaLaboralTable_str.copy()

    HistoriaLaboralTable_str['Fecha de Movimiento'] = [format_spanish_date(date, date_type="format_string") for date in FechaMovimiento]
    HistoriaLaboralTable_num['Sueldo'] = Sueldo
    return HistoriaLaboralTable_str, HistoriaLaboralTable_num

HistoriaLaboralTable_str, HistorialLaboralTable_num = HistorialLaboralTabla_fcn(texto)

display(HistorialLaboralTable_num.head())
display(HistoriaLaboralTable_str.head())

def HistorialLaboralDesglosada_fcn(HistoriaLaboralTable, sigueCotizando, FechasUltimaBaja):
    Bajas = np.where(HistoriaLaboralTable['Movimiento'] == 'BAJA')[0]
    FechaFinal = HistoriaLaboralTable['Fecha de Movimiento'].values
    # We will apply the convertir_a_fecha function to the list FechaFinal
    FechaFinal = [format_spanish_date(fecha, date_type="format_date") for fecha in FechaFinal]

    for idx in range(len(HistoriaLaboralTable)):
        if idx not in Bajas:
            FechaFinal[idx] =  FechaFinal[idx] - timedelta(days=1)

    FechaFinal = np.roll(FechaFinal, 1)
    if sigueCotizando:
        FechaFinal[0] = FechasUltimaBaja
#
    HistoriaLaboralDesglosada = HistoriaLaboralTable.copy()
    HistoriaLaboralDesglosada['Fecha Final'] = FechaFinal
    HistoriaLaboralDesglosada.drop(Bajas, inplace=True)
    HistoriaLaboralDesglosada = HistoriaLaboralDesglosada.rename(columns={"Fecha de Movimiento": "Fecha Inicial"})

    HistoriaLaboralDesglosada["Fecha Inicial"] = [format_spanish_date(fecha, date_type="format_date") for fecha in HistoriaLaboralDesglosada["Fecha Inicial"]]
    HistoriaLaboralDesglosada["Fecha Inicial"] = HistoriaLaboralDesglosada["Fecha Inicial"] + timedelta(days = 1)

#     # Move the column "FechaFinal" to be after "FechaInicial"
    cols = list(HistoriaLaboralDesglosada.columns)
    cols.insert(cols.index("Fecha Inicial") + 1, cols.pop(cols.index("Fecha Final")))
    HistoriaLaboralDesglosada = HistoriaLaboralDesglosada[cols]

    HistoriaLaboralDesglosada_str = HistoriaLaboralDesglosada.copy()
    HistoriaLaboralDesglosada_str['Fecha Inicial'] = [format_spanish_date(str(date), date_type="format_string") for date in HistoriaLaboralDesglosada['Fecha Inicial']]
    HistoriaLaboralDesglosada_str['Fecha Final'] = [format_spanish_date(str(date), date_type="format_string") for date in HistoriaLaboralDesglosada['Fecha Final']]
    HistoriaLaboralDesglosada_str['Sueldo'] = [convert_double_currency(float(valor)) for valor in HistoriaLaboralDesglosada['Sueldo']]

    return HistoriaLaboralDesglosada, HistoriaLaboralDesglosada_str
#
HistoriaLaboralTable, HistorialLaboralTable_str = HistorialLaboralDesglosada_fcn(HistorialLaboralTable_num, sigueCotizando, fechasUltimaBaja)

display(HistoriaLaboralTable.head())
display(HistorialLaboralTable_str.head())

,Movimiento,Fecha de Movimiento,Sueldo,Empleador,Entidad Federativa
0,BAJA,30/09/2020,128.79,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,MODIFICACION DE SALARIO,01/01/2020,128.79,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,MODIFICACION DE SALARIO,01/01/2019,107.32,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
3,MODIFICACION DE SALARIO,01/12/2017,92.35,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
4,MODIFICACION DE SALARIO,01/01/2017,83.66,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS


,Movimiento,Fecha de Movimiento,Sueldo,Empleador,Entidad Federativa
0,BAJA,"Miércoles, 30 de septiembre de 2020",$128.79,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
1,MODIFICACION DE SALARIO,"Miércoles, 1 de enero de 2020",$128.79,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,MODIFICACION DE SALARIO,"Martes, 1 de enero de 2019",$107.32,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
3,MODIFICACION DE SALARIO,"Viernes, 1 de diciembre de 2017",$92.35,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
4,MODIFICACION DE SALARIO,"Domingo, 1 de enero de 2017",$83.66,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS


,Movimiento,Fecha Inicial,Fecha Final,Sueldo,Empleador,Entidad Federativa
1,MODIFICACION DE SALARIO,2020-01-02,2020-09-30,128.79,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,MODIFICACION DE SALARIO,2019-01-02,2019-12-31,107.32,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
3,MODIFICACION DE SALARIO,2017-12-02,2018-12-31,92.35,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
4,MODIFICACION DE SALARIO,2017-01-02,2017-11-30,83.66,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
5,MODIFICACION DE SALARIO,2016-04-02,2016-12-31,80.0,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS


,Movimiento,Fecha Inicial,Fecha Final,Sueldo,Empleador,Entidad Federativa
1,MODIFICACION DE SALARIO,"Jueves, 2 de enero de 2020","Miércoles, 30 de septiembre de 2020",$128.79,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
2,MODIFICACION DE SALARIO,"Miércoles, 2 de enero de 2019","Martes, 31 de diciembre de 2019",$107.32,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
3,MODIFICACION DE SALARIO,"Sábado, 2 de diciembre de 2017","Lunes, 31 de diciembre de 2018",$92.35,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
4,MODIFICACION DE SALARIO,"Lunes, 2 de enero de 2017","Jueves, 30 de noviembre de 2017",$83.66,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS
5,MODIFICACION DE SALARIO,"Sábado, 2 de abril de 2016","Sábado, 31 de diciembre de 2016",$80.00,CARLOS DAVID ALFONZO UTRILLA,CHIAPAS


# Tabla de Salarios


In [11]:
def salario_promedio_250tabla(HistoriaLaboralDesglosada):
    salario_promedio_250 = HistoriaLaboralDesglosada[["Fecha Inicial", "Fecha Final", "Sueldo"]]
    salario_promedio_250 = salario_promedio_250.sort_values(by=["Fecha Inicial", "Fecha Final"], ascending=[False, False])

    salario_promedio_250_str = salario_promedio_250.copy()
    salario_promedio_250_str['Fecha Inicial'] = [format_spanish_date(str(date), date_type="format_string") for date in salario_promedio_250['Fecha Inicial']]
    salario_promedio_250_str['Fecha Final'] = [format_spanish_date(str(date), date_type="format_string") for date in salario_promedio_250['Fecha Final']]
    salario_promedio_250_str['Sueldo'] = [convert_double_currency(float(valor)) for valor in salario_promedio_250['Sueldo']]

    return salario_promedio_250, salario_promedio_250_str

tabla_salarios_date, tabla_salarios_str = salario_promedio_250tabla(HistoriaLaboralTable)
display(tabla_salarios_date.head())
display(tabla_salarios_str.head())

,Fecha Inicial,Fecha Final,Sueldo
1,2020-01-02,2020-09-30,128.79
2,2019-01-02,2019-12-31,107.32
3,2017-12-02,2018-12-31,92.35
4,2017-01-02,2017-11-30,83.66
5,2016-04-02,2016-12-31,80.0


,Fecha Inicial,Fecha Final,Sueldo
1,"Jueves, 2 de enero de 2020","Miércoles, 30 de septiembre de 2020",$128.79
2,"Miércoles, 2 de enero de 2019","Martes, 31 de diciembre de 2019",$107.32
3,"Sábado, 2 de diciembre de 2017","Lunes, 31 de diciembre de 2018",$92.35
4,"Lunes, 2 de enero de 2017","Jueves, 30 de noviembre de 2017",$83.66
5,"Sábado, 2 de abril de 2016","Sábado, 31 de diciembre de 2016",$80.00


# Salarios Modificados (Modalidad 40)

In [12]:
fecha_inicio = pd.to_datetime('2020-10-01').date()
fecha_final = pd.to_datetime('2023-10-31').date()
salario = 2172.0

def tabla_salarios_modificada(tabla_salarios_original,fecha_inicio, fecha_final, salario):
    nueva_fila = {'Fecha Inicial': fecha_inicio, 'Fecha Final': fecha_final, 'Sueldo': salario}
    nueva_tabla = pd.concat([tabla_salarios_original, pd.DataFrame([nueva_fila])], ignore_index=True)
    # We will sort by the column "Fecha Inicial"
    nueva_tabla = nueva_tabla.sort_values(by=["Fecha Inicial"], ascending=[False])
    # We will change the date format for the first column
    nueva_tabla['Fecha Inicial'] = pd.to_datetime(nueva_tabla['Fecha Inicial'], format='%d/%m/%Y').dt.date
    # we will change the date format for the second column, from day/month/year to year-month-day
    nueva_tabla['Fecha Final'] = pd.to_datetime(nueva_tabla['Fecha Final'], format='%d/%b/%Y').dt.date

    nueva_tabla_str = nueva_tabla.copy()
    nueva_tabla_str['Fecha Inicial'] = [format_spanish_date(str(date), date_type="format_string") for date in nueva_tabla['Fecha Inicial']]
    nueva_tabla_str['Fecha Final'] = [format_spanish_date(str(date), date_type="format_string") for date in nueva_tabla['Fecha Final']]
    nueva_tabla_str['Sueldo'] = [convert_double_currency(float(valor)) for valor in nueva_tabla['Sueldo']]
    return nueva_tabla, nueva_tabla_str

nueva_tabla_salarios, nueva_tabla_salarios_str = tabla_salarios_modificada(tabla_salarios_original=tabla_salarios_date,
                          fecha_inicio=fecha_inicio,
                          fecha_final=fecha_final,
                          salario=salario)

display(nueva_tabla_salarios_str.head())

,Fecha Inicial,Fecha Final,Sueldo
16,"Jueves, 1 de octubre de 2020","Martes, 31 de octubre de 2023","$2,172.00"
0,"Jueves, 2 de enero de 2020","Miércoles, 30 de septiembre de 2020",$128.79
1,"Miércoles, 2 de enero de 2019","Martes, 31 de diciembre de 2019",$107.32
2,"Sábado, 2 de diciembre de 2017","Lunes, 31 de diciembre de 2018",$92.35
3,"Lunes, 2 de enero de 2017","Jueves, 30 de noviembre de 2017",$83.66


## Tabla de salarios promedios para el cálculo de las últimas 250 semanas



In [13]:
def salario_promedio_fcn(semanas_contar, semanas_reconocidas, tabla_salarios):
    if semanas_contar == 0:
        return 0, pd.DataFrame()
  # Reset the index of salario_promedio_250 to ensure it starts from 0
    tabla_salarios = tabla_salarios.reset_index(drop=True)
    # Convert Fecha Inicial and Fecha Final columns to datetime.date
    tabla_salarios['Fecha Inicial'] = pd.to_datetime(tabla_salarios['Fecha Inicial']).dt.date
    tabla_salarios['Fecha Final'] = pd.to_datetime(tabla_salarios['Fecha Final']).dt.date
    # Initialize necessary variables
    salario_acumulado_periodo = np.zeros(len(tabla_salarios['Fecha Inicial']))
    fecha_superior = tabla_salarios['Fecha Final']
    fecha_inferior = tabla_salarios['Fecha Inicial']
    salario_diario =  np.array(tabla_salarios['Sueldo']).astype(float)
    semanas_periodo = salario_acumulado_periodo.copy()
    fecha_corte = [None] * len(fecha_superior)
    semanas_acumuladas = salario_acumulado_periodo.copy()
    semanas_cuenta = salario_acumulado_periodo.copy()

    for idx in range(len(fecha_superior)):
        # Calculate weeks in each period
        semanas_periodo[idx] = (fecha_superior[idx] - fecha_inferior[idx]).days / 7 + 1 / 7
        if idx == 0:
            if semanas_periodo[idx] <= semanas_contar:
                fecha_corte[idx] = fecha_inferior[idx]
            else:
                fecha_corte[idx] = fecha_superior[idx] - timedelta(weeks=semanas_contar)

            semanas_cuenta[idx] = (fecha_superior[idx] - fecha_inferior[idx]).days / 7 + 1 / 7
            semanas_acumuladas[idx] = semanas_periodo[idx]
        else:
            semanas_acumuladas[idx] = semanas_periodo[idx]

            if semanas_acumuladas[idx - 1] < semanas_contar:
                if semanas_acumuladas[idx - 1] + semanas_periodo[idx] < semanas_contar:
                    if fecha_corte[idx - 1] is None:  # Check if previous fecha_corte is None
                        fecha_corte[idx] = fecha_inferior[idx]  # Use fecha_inferior if previous is None
                    else:
                        fecha_corte[idx] = min(fecha_corte[idx - 1], fecha_inferior[idx])  # Use min if previous is not None

                else:
                    fecha_corte[idx] = fecha_superior[idx] - timedelta(weeks=(semanas_contar - semanas_acumuladas[idx - 1]))
            else:
                fecha_corte[idx] = fecha_corte[idx - 1]

            if fecha_corte[idx] > fecha_superior[idx]:
                semanas_cuenta[idx] = 0
            else:
                semanas_cuenta[idx] = (fecha_superior[idx] - max(fecha_corte[idx], fecha_inferior[idx])).days / 7 + 2 / 7

            if semanas_acumuladas[idx - 1] == semanas_contar:
                semanas_acumuladas[idx] = semanas_contar
            else:
                if semanas_acumuladas[idx - 1] + semanas_cuenta[idx] > semanas_contar:
                    semanas_acumuladas[idx] = semanas_contar
                else:
                    if fecha_corte[idx] != fecha_inferior[idx]:
                        semanas_acumuladas[idx] = semanas_acumuladas[idx - 1] + (max(fecha_corte[idx - 1], fecha_superior[idx]) - fecha_corte[idx]).days / 7 + 1 / 7
                    else:
                        semanas_acumuladas[idx] = semanas_acumuladas[idx - 1] + (min(fecha_corte[idx - 1], fecha_superior[idx]) - fecha_corte[idx]).days / 7 + 1 / 7

        salario_acumulado_periodo[idx] = salario_diario[idx] * 7 * semanas_cuenta[idx] if semanas_cuenta[idx] > 0 else 0

    tabla_salario_promedio = pd.DataFrame({
        'Fecha Inicio': fecha_inferior,
        'Fecha Final': fecha_superior,
        # Convert to string with currency format
        'Salario Diario': salario_diario,
        'Semanas totales en el periodo': np.round(semanas_periodo, 1),
        'Fecha de Corte': pd.to_datetime(fecha_corte),
        'Semanas tomadas en cuenta': np.round(semanas_cuenta, 1),
        'Salario Acumulado en el Periodo': salario_acumulado_periodo,
        'Semanas Acumuladas Totales': np.round(semanas_acumuladas, 1)
    })
    tabla_salario_promedio['Fecha de Corte'] = tabla_salario_promedio['Fecha de Corte'].dt.date
    salario_acumulado = np.sum(salario_acumulado_periodo)

    if semanas_reconocidas < semanas_contar:
        salario_promedio_diario = salario_acumulado / (semanas_reconocidas * 7)
    else:
        salario_promedio_diario = salario_acumulado / (semanas_contar * 7)

    tabla_salario_promedio_str = tabla_salario_promedio.copy()
    tabla_salario_promedio_str['Salario Diario'] = [convert_double_currency(value) for value in tabla_salario_promedio_str['Salario Diario']]
    tabla_salario_promedio_str['Salario Acumulado en el Periodo'] = [convert_double_currency(value) for value in tabla_salario_promedio_str['Salario Acumulado en el Periodo']]
    tabla_salario_promedio_str['Fecha Inicio'] = [format_spanish_date(str(date), date_type="format_string") for date in tabla_salario_promedio_str['Fecha Inicio']]
    tabla_salario_promedio_str['Fecha Final'] = [format_spanish_date(str(date), date_type="format_string") for date in tabla_salario_promedio_str['Fecha Final']]

    return salario_promedio_diario, tabla_salario_promedio, tabla_salario_promedio_str

# Example usage
SEMANAS_CONTAR = 250
SemanasReconocidas = Usuario.semanas_totales
SALARIO_PROMEDIO_DIARIO, tabla_salario_promedio_num, tabla_salario_promedio_str = salario_promedio_fcn(semanas_contar=SEMANAS_CONTAR,
                                                                                                        semanas_reconocidas=SemanasReconocidas,
                                                                                                       tabla_salarios=tabla_salarios_date)

display(tabla_salario_promedio_num.head())
display(tabla_salario_promedio_str.head())
print(f"Salario Promedio:\n{round(SALARIO_PROMEDIO_DIARIO, 1)}")

,Fecha Inicio,Fecha Final,Salario Diario,Semanas totales en el periodo,Fecha de Corte,Semanas tomadas en cuenta,Salario Acumulado en el Periodo,Semanas Acumuladas Totales
0,2020-01-02,2020-09-30,128.79,39.0,2020-01-02,39.0,35159.67,39.0
1,2019-01-02,2019-12-31,107.32,52.0,2019-01-02,52.1,39171.80,91.0
2,2017-12-02,2018-12-31,92.35,56.4,2017-12-02,56.6,36570.60,147.4
3,2017-01-02,2017-11-30,83.66,47.6,2017-01-02,47.7,27942.44,195.0
4,2016-04-02,2016-12-31,80.00,39.1,2016-04-02,39.3,22000.00,234.1


,Fecha Inicio,Fecha Final,Salario Diario,Semanas totales en el periodo,Fecha de Corte,Semanas tomadas en cuenta,Salario Acumulado en el Periodo,Semanas Acumuladas Totales
0,"Jueves, 2 de enero de 2020","Miércoles, 30 de septiembre de 2020",$128.79,39.0,2020-01-02,39.0,"$35,159.67",39.0
1,"Miércoles, 2 de enero de 2019","Martes, 31 de diciembre de 2019",$107.32,52.0,2019-01-02,52.1,"$39,171.80",91.0
2,"Sábado, 2 de diciembre de 2017","Lunes, 31 de diciembre de 2018",$92.35,56.4,2017-12-02,56.6,"$36,570.60",147.4
3,"Lunes, 2 de enero de 2017","Jueves, 30 de noviembre de 2017",$83.66,47.6,2017-01-02,47.7,"$27,942.44",195.0
4,"Sábado, 2 de abril de 2016","Sábado, 31 de diciembre de 2016",$80.00,39.1,2016-04-02,39.3,"$22,000.00",234.1


Salario Promedio:
96.8


In [14]:
SALARIO_PROMEDIO_DIARIO, tabla_salario_promedio_num, tabla_salario_promedio_str = salario_promedio_fcn(semanas_contar=SEMANAS_CONTAR,
                                                                                                        semanas_reconocidas=SemanasReconocidas,
                                                                                                       tabla_salarios=nueva_tabla_salarios)

display(tabla_salario_promedio_num.head())
display(tabla_salario_promedio_str.head())
print(f"Salario Promedio:\n{round(SALARIO_PROMEDIO_DIARIO, 1)}")

,Fecha Inicio,Fecha Final,Salario Diario,Semanas totales en el periodo,Fecha de Corte,Semanas tomadas en cuenta,Salario Acumulado en el Periodo,Semanas Acumuladas Totales
0,2020-10-01,2023-10-31,2172.00,160.9,2020-10-01,160.9,2445672.00,160.9
1,2020-01-02,2020-09-30,128.79,39.0,2020-01-02,39.1,35288.46,199.9
2,2019-01-02,2019-12-31,107.32,52.0,2019-01-14,50.4,37883.96,250.0
3,2017-12-02,2018-12-31,92.35,56.4,2019-01-14,0.0,0.00,250.0
4,2017-01-02,2017-11-30,83.66,47.6,2019-01-14,0.0,0.00,250.0


,Fecha Inicio,Fecha Final,Salario Diario,Semanas totales en el periodo,Fecha de Corte,Semanas tomadas en cuenta,Salario Acumulado en el Periodo,Semanas Acumuladas Totales
0,"Jueves, 1 de octubre de 2020","Martes, 31 de octubre de 2023","$2,172.00",160.9,2020-10-01,160.9,"$2,445,672.00",160.9
1,"Jueves, 2 de enero de 2020","Miércoles, 30 de septiembre de 2020",$128.79,39.0,2020-01-02,39.1,"$35,288.46",199.9
2,"Miércoles, 2 de enero de 2019","Martes, 31 de diciembre de 2019",$107.32,52.0,2019-01-14,50.4,"$37,883.96",250.0
3,"Sábado, 2 de diciembre de 2017","Lunes, 31 de diciembre de 2018",$92.35,56.4,2019-01-14,0.0,$0.00,250.0
4,"Lunes, 2 de enero de 2017","Jueves, 30 de noviembre de 2017",$83.66,47.6,2019-01-14,0.0,$0.00,250.0


Salario Promedio:
1439.3


## Cálculo de Obtención de Pensión con entrada Manual

In [15]:
edad_pension = 62 #@param {type:"integer"}
year = 2023 #@param {type:"integer"}
semanas_reconocidas = 966 #@param {type:"integer"}
salario_promedio = 1439.16 #@param {type:"number"}
porcentaje_asignaciones = 0.15 #@param {type:"number"}

class PensionManual:
    def __init__(self, edad_pension, year, semanas_reconocidas, salario_promedio, porcentaje_asignaciones):
        self.edad_pension = edad_pension
        self.year = year
        self.semanas_reconocidas = semanas_reconocidas
        self.salario_promedio = salario_promedio
        self.porcentaje_asignaciones = porcentaje_asignaciones
        self.porcentaje_asignaciones_string = f"{round(porcentaje_asignaciones * 100, 1)}%"
        self.tabla_salarios_minimos = pd.read_csv('TABLAS/TABLA_SALARIOSMINIMOS.txt', sep='\t')
        self.tabla_porcentaje_edad = pd.read_csv('TABLAS/TABLA_PORCENTAJE_EDAD.txt', sep='\t')
        self.tabla_uma = pd.read_csv('TABLAS/TABLA_UMA.txt', sep='\t')
        self.tabla_relacion_umas = pd.read_csv('TABLAS/TABLA_RELACION_UMAS_SALARIO_MINIMO.txt', sep='\t')

    def salario_minimo_garantizado_fcn(self):
            tabla = self.tabla_salarios_minimos
            out = tabla[tabla.iloc[:, 0] == year]
            salario_minimo_ano_actual = out.iloc[0, 1]#.strip("$")
            salario_minimo_mensual = out.iloc[0, 4]
            pension_minima_garantizada = out.iloc[0, -1]
            salario_maximo_asignado_diario = out.iloc[0, 6]

            print(f"Los siguientes datos son para el año: {self.year}")

            # We will also create a pandas dataframe
            df = pd.DataFrame({
                # 'Año': year,
                'Salario Minimo Diario': salario_minimo_ano_actual,
                'Salario Minimo Mensual': salario_minimo_mensual,
                'Pension Mínima Garantizada': pension_minima_garantizada,
                'Salario Máximo Asignado Diario': salario_maximo_asignado_diario
            }, index=[self.year])
            display(df)

            self.salario_minimo_ano_actual = salario_minimo_ano_actual
            self.salario_minimo_mensual = salario_minimo_mensual
            self.pension_minima_garantizada = pension_minima_garantizada
            self.salario_maximo_asignado_diario = salario_maximo_asignado_diario

            return salario_minimo_ano_actual, salario_minimo_mensual, pension_minima_garantizada, salario_maximo_asignado_diario, df

    def porcentaje_vejez_fcn(self):
        tabla = self.tabla_porcentaje_edad
        idx = tabla[tabla['EDAD'] == min(self.edad_pension, 65)]['% PENSIÓN'] # Finds the position of the table with the target age
        perc_string = idx.iloc[0] # Obtain the percentage by multiplying by 100%
        porcentaje_vejez =float( perc_string.strip('%')) / 1e2
        self.porcentaje_vejez = porcentaje_vejez
        self.porcentaje_vejez_string = perc_string

    def valor_uma_actual_year_fcn(self):
        tabla = self.tabla_uma
        idx = tabla[tabla['AÑO'] == self.year]['VALOR $'] # Finds the position of the table with the target year
        valor_uma_string = idx.iloc[0]
        valor_uma_numerico =float( valor_uma_string.strip('$'))
        self.valor_uma_numerico = valor_uma_numerico
        self.valor_uma_string = valor_uma_string

    def relacion_uma_salario_fcn(self):
        relacion_salario_uma = round(self.salario_promedio / self.valor_uma_numerico, 2)
        self.relacion_uma_salario = relacion_salario_uma

    def porcentaje_cuantia_fcn(self):
        tabla = self.tabla_relacion_umas
        #subs substracts the variable from all values and then we take only those values inferior, finally we take the pos. of the largest value
        closest_idx = tabla[tabla["Inferior"].sub(self.relacion_uma_salario) < 0]['Inferior'].idxmax()
        out_string = tabla.iloc[closest_idx, 2] # Pos. 2 relates to Porcentaje cuantía básica [%]
        porcentaje_cuantia = float(out_string.strip(('%')))/1e2
        self.porcentaje_cuantia = porcentaje_cuantia
        # self.porcentaje_cuantia_string = out_string

    def semanas_equivalentes_year_fcn(self):
        semanas_despues500 = semanas_reconocidas - 500
        upper_bound = 0.5
        lower_bound = 0.25
        if lower_bound < abs(semanas_despues500/52 - np.fix(semanas_despues500/52)) <= upper_bound:
            SEMANAS_EQUIVALENTES = np.fix(semanas_despues500 / 52) + 0.5
        else:
            SEMANAS_EQUIVALENTES = np.round(semanas_despues500/52, decimals=0)
        self.semanas_equivalentes = SEMANAS_EQUIVALENTES

    def salario_incremento_anual_perc_fcn(self):
        tabla = self.tabla_relacion_umas
        closest_idx = (tabla['Porcentaje cuantía básica \r\n[%]'].str.replace('%', '', regex=True)).astype(float)
        closest_idx = closest_idx.sub(self.porcentaje_cuantia*1e2) <= 0
        closest_idx = tabla[closest_idx]['Salario incremento anual [%]'].idxmax()
        salario_incremento_anual_perc = tabla.loc[closest_idx, 'Salario incremento anual [%]']
        salario_incremento_anual_perc = float(salario_incremento_anual_perc.strip('%')) / 1e2
        self.salario_incremento_anual_perc = salario_incremento_anual_perc

    def incremento_perc_fcn(self):
        incremento_perc = round(self.salario_incremento_anual_perc * self.semanas_equivalentes, 4)
        self.incremento_perc = incremento_perc
        self.incremento_perc_string = f'{incremento_perc*1e2:.2f}%'
        return incremento_perc

    def incremento_dinero_fcn(self):
        incremento_dinero = round(self.incremento_perc * self.salario_promedio * 1.11 * 365, 2)
        self.incremento_dinero = incremento_dinero

    def cuantia_basica_anual_fcn(self):
        cuantia_basica_anual = round(self.porcentaje_cuantia * self.salario_promedio * 365 * 1.11, 2)
        self.cuantia_basica_anual = cuantia_basica_anual

    def cuantia_vejez_fcn(self):
        cuantia_vejez = self.cuantia_basica_anual + self.incremento_dinero
        self.cuantia_vejez = cuantia_vejez

    def pension_cesantia_edad_avanzada_fcn(self):
        pension_cesantia_edad_avanzada = round(self.cuantia_vejez * self.porcentaje_vejez, 2)
        self.pension_cesantia_edad_avanzada = pension_cesantia_edad_avanzada

    def importe_asignaciones_familiares_fcn(self):
        importe_asignaciones_familiares = round(self.porcentaje_asignaciones * self.pension_cesantia_edad_avanzada, 2)
        self.importe_asignaciones_familiares = importe_asignaciones_familiares

    def importe_anual_cesantia_fcn(self):
        importe_anual_cesantia = round(self.importe_asignaciones_familiares + self.pension_cesantia_edad_avanzada, 2)
        self.importe_anual_cesantia = importe_anual_cesantia

    def importe_mensual_calculado_fcn(self):
        importe_mensual_calculado = round(self.importe_anual_cesantia / 12, 1)
        self.importe_mensual_calculado = importe_mensual_calculado

    def pension_minima_garantizada_fcn(self):
        pension_minima_garantizada = self.pension_minima_garantizada.replace('$', '').replace(',', '')
        pension_minima_garantizada = float(pension_minima_garantizada)
        self.pension_minima_garantizada = pension_minima_garantizada

    def pension_final_fcn(self):
        pension_final = round(max(self.importe_mensual_calculado, self.pension_minima_garantizada), 2)
        self.pension_final = pension_final

    def calculo_pension(self):
        self.salario_minimo_garantizado_fcn()
        self.porcentaje_vejez_fcn()
        self.valor_uma_actual_year_fcn()
        self.relacion_uma_salario_fcn()
        self.porcentaje_cuantia_fcn()
        self.semanas_equivalentes_year_fcn()
        self.salario_incremento_anual_perc_fcn()
        self.incremento_perc_fcn()
        self.incremento_dinero_fcn()
        self.cuantia_basica_anual_fcn()
        self.cuantia_vejez_fcn()
        self.pension_cesantia_edad_avanzada_fcn()
        self.importe_asignaciones_familiares_fcn()
        self.importe_anual_cesantia_fcn()
        self.importe_mensual_calculado_fcn()
        self.pension_minima_garantizada_fcn()
        self.pension_final_fcn()

        pension_df = pd.DataFrame({'Pensión Final': [convert_double_currency(self.pension_final)],
                                   'Semanas Reconocidas': [self.semanas_reconocidas],
                                   'Salario Promedio': [convert_double_currency(self.salario_promedio)]
                                   })

        detalles_df = pd.DataFrame({
                                   'Porcentaje Asignaciones Familiares': [self.porcentaje_asignaciones_string],
                                   'Porcentaje Vejez': [self.porcentaje_vejez_string],
                                   'Valor UMA Actual Year': [self.valor_uma_string],
                                   'Incremento %': [self.incremento_perc_string],
                                   'Incremento Dinero': [convert_double_currency(self.incremento_dinero)],
                                   'Cuantia Basica Anual': [convert_double_currency(self.cuantia_basica_anual)],
                                   'Cuantia Vejez': [self.cuantia_vejez],
                                   'Pension Cesantia Edad Avanzada': [convert_double_currency(self.pension_cesantia_edad_avanzada)],
                                   'Importe Asignaciones Familiares': [convert_double_currency(self.importe_asignaciones_familiares)],
                                   'Importe Anual Cesantia': [convert_double_currency(self.importe_anual_cesantia)],
                                   'Importe Mensual Calculado': [convert_double_currency(self.importe_mensual_calculado)],
                                   'Pension Minima Garantizada': [convert_double_currency(self.pension_minima_garantizada)],
                                   'Pension Final': [convert_double_currency(self.pension_final)]
                                   })

        detalles_df = round(detalles_df.T, 2)
        # We will rename the column
        detalles_df.columns = ['Valores']

        return pension_df, detalles_df

# Example Usage
pension_manual = PensionManual(edad_pension=edad_pension, year=year, semanas_reconocidas=semanas_reconocidas, salario_promedio=salario_promedio, porcentaje_asignaciones=porcentaje_asignaciones)


pension_df, detalles_df = pension_manual.calculo_pension()
display(pension_df)
display(detalles_df)

Los siguientes datos son para el año: 2023


,Salario Minimo Diario,Salario Minimo Mensual,Pension Mínima Garantizada,Salario Máximo Asignado Diario
2023,$207.44,"$6,309.63","$7,003.69","$2,593.50"


,Pensión Final,Semanas Reconocidas,Salario Promedio
0,"$16,647.50",966,"$1,439.16"


,Valores
Porcentaje Asignaciones Familiares,15.0%
Porcentaje Vejez,85%
Valor UMA Actual Year,$103.74
Incremento %,22.05%
Incremento Dinero,"$128,568.19"
Cuantia Basica Anual,"$75,799.84"
Cuantia Vejez,204368.03
Pension Cesantia Edad Avanzada,"$173,712.83"
Importe Asignaciones Familiares,"$26,056.92"
Importe Anual Cesantia,"$199,769.75"


# Fechas de Pago Modalidad 40

In [16]:
def pagos40(salario, start_date, durata):
    tabla_uma = pd.read_csv('TABLAS/TABLA_UMA.txt', sep='\t')
    TABLA_AUMENTO_PAGOMOD40 = pd.read_csv('TABLAS/TABLA_AUMENTOPAGO40.txt', sep='\t')
    TABLA_SALARIOSMINIMOS = pd.read_csv('TABLAS/TABLA_SALARIOSMINIMOS.txt', sep='\t')
    CUOTA_OBRERA = 6.925 / 1e2;

    salario_array = np.full(durata + 1, salario)  # Creates an array with the same value
    second_date = start_date + relativedelta(months=1)
    second_date = second_date.replace(day=1)

    monthly_series = pd.date_range(start=second_date, periods=durata, freq='MS')  # MS: Month Start
    time_series_df = pd.concat([pd.Series(start_date), pd.Series(monthly_series)])
    time_series_df = pd.to_datetime(time_series_df)

    salarios_minimos_array = list()
    salarios_maximos_array = list()
    salario_uma40_array = list()
    DiasCubiertosMensuales_array = list()
    PORCENTAJE_PAGO_array = list()

    def money_to_float(input_str: str, *args):
        for chars in args:
            input_str = input_str.replace(chars, '')
        return float(input_str.strip())

    # Wtih applymap we apply our custom function to each element specified in the iloc
    # indexation, we remove the '%' with the replace function and divide by 1e2
    TABLA_AUMENTO_PAGOMOD40.iloc[:, 1:] = TABLA_AUMENTO_PAGOMOD40.iloc[:, 1:].map(
        lambda x: money_to_float(x, '%') / 1e2)
    TABLA_AUMENTO_PAGOMOD40 = TABLA_AUMENTO_PAGOMOD40.set_index('AÑO')
    salarios_minimos = TABLA_SALARIOSMINIMOS.loc[:, 'SM DIARIO ($)'].map(lambda x: money_to_float(x, '$'))
    salarios_maximos = TABLA_SALARIOSMINIMOS.iloc[:, 6].map(lambda x: money_to_float(x, '$', ','))

    for i in range(durata + 1):
        if time_series_df.iloc[i].month == 1:
            year_considered = time_series_df.iloc[i].year - 1
        else:
            year_considered = time_series_df.iloc[i].year
        # Find the index of rows where the 'AÑO' column equals 'year_considered'
        row = tabla_uma[tabla_uma['AÑO'] == year_considered].index
        row = row.tolist()[0]
        perc_row = tabla_uma.index[tabla_uma['AÑO'] == time_series_df.iloc[i].year].tolist()[0]
        salarios_minimos_array.append(salarios_minimos[tabla_uma['AÑO'] == time_series_df.iloc[i].year].tolist()[0])
        salarios_maximos_array.append(salarios_maximos[tabla_uma['AÑO'] == time_series_df.iloc[i].year].tolist()[0])
        salario_array[i] = min(salarios_maximos_array[i], max(salarios_minimos_array[i], salario_array[i]))

        uma_row = tabla_uma.iloc[perc_row, 1]
        uma_row = uma_row.replace('$', '')
        uma_row = float(uma_row)
        salario_uma40_array.append(round(salario_array[i] / uma_row, 1))

        #Find the last day of the current month
        lastDayOfMonth = time_series_df.iloc[i].replace(day=1) + relativedelta(
            day=31)  # Even if the month does not have 31 days, the relative delta manages
        # Calculate the number of remaining days
        DiasCubiertosMensuales_array.append((lastDayOfMonth - time_series_df.iloc[i]).days + 1)

        if salario_uma40_array[i] >= 4.01:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -2]
        elif salario_uma40_array[i] >= 3.51:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -3]
        elif salario_uma40_array[i] >= 3:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -4]
        elif salario_uma40_array[i] >= 2.51:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -5]
        elif salario_uma40_array[i] >= 2:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -6]
        elif salario_uma40_array[i] >= 1.51:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -7]
        elif salario_uma40_array[i] >= 1:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -8]
        else:
            CUOTA_PATRONAL = TABLA_AUMENTO_PAGOMOD40.iloc[perc_row, -9]
        PORCENTAJE_PAGO_array.append(CUOTA_PATRONAL + CUOTA_OBRERA)

    PORCENTAJE_PAGO_array_string = map(lambda x: round(x * 1e2, 2) / 1e2, PORCENTAJE_PAGO_array)
    # Wtih map we apply our custom function to each element specified in the iloc
    # indexation, we remove the '%' with the replace function and divide by 1e2
    time_series_df = time_series_df.map(lambda x: x.strftime('%Y-%m-%d'))
    salario_asignado_mensual = salario_array * DiasCubiertosMensuales_array
    mes_de_pago = time_series_df.map(lambda x: pd.to_datetime(x).month)
    mes_de_pago.replace(
        {1: 'Enero', 2: 'Febrero', 3: 'Marzo', 4: 'Abril', 5: 'Mayo', 6: 'Junio', 7: 'Julio', 8: 'Agosto',
         9: 'Septiembre', 10: 'Octubre', 11: 'Noviembre', 12: 'Diciembre'}, inplace=True)

    # # Pago Mensual
    PAGO_MENSUAL_array = (salario_asignado_mensual * PORCENTAJE_PAGO_array).round(2)
    pago_acumulado = np.cumsum(PAGO_MENSUAL_array)
    tabla_pago40 = pd.DataFrame({'Número de Pago': np.arange(1, durata + 2),
                                 'Fechas de Pago': time_series_df,
                                 'Día de Inicio de Cobertura': time_series_df.map(lambda x: pd.to_datetime(x).day),
                                 'Mes de Pago': mes_de_pago,
                                 'Año de Pago': time_series_df.map(lambda x: pd.to_datetime(x).year),
                                 'Salario Diario Asignado': salario_array,
                                 'Relación Salario/UMA': salario_uma40_array,
                                 'Días cubiertos': DiasCubiertosMensuales_array,
                                 'Salario Asignado Mensual': salario_asignado_mensual,
                                 '% de pago respecto del salario asignado': PORCENTAJE_PAGO_array_string,
                                 'Pago Mensual': PAGO_MENSUAL_array,
                                 'Pago Acumulado': pago_acumulado,
                                 'Salario Mínimo del año en curso': salarios_minimos_array,
                                 'Salario Máximo del año en curso': salarios_maximos_array})

    tabla_pago40.set_index(np.arange(1, durata + 2), inplace=True)
    tabla_pago40_string = tabla_pago40.copy()
    tabla_pago40_string[['Salario Diario Asignado', 'Salario Asignado Mensual', 'Pago Mensual', 'Pago Acumulado',
                         'Salario Mínimo del año en curso', 'Salario Máximo del año en curso']] = tabla_pago40[
        ['Salario Diario Asignado', 'Salario Asignado Mensual', 'Pago Mensual', 'Pago Acumulado',
         'Salario Mínimo del año en curso', 'Salario Máximo del año en curso']].map("${:,.2f}".format)
    tabla_pago40_string['% de pago respecto del salario asignado'] = (
    tabla_pago40['% de pago respecto del salario asignado']).map("{:.2%}".format)
    tabla_pago40_string['Fechas de Pago'] = [format_spanish_date(date, date_type="format_string") for date in tabla_pago40['Fechas de Pago']]

    return tabla_pago40, tabla_pago40_string

In [17]:
salario = 2593.5 #@param {type:"number"}
start_date = "2023-02-03" #@param {type:"date"}
start_date = datetime.strptime(start_date, "%Y-%m-%d").date()
durata = 24 #@param {type:"integer"}

pagos_num, pagos_str = pagos40(salario, start_date, durata)
print("Numerical")
display(pagos_num.head(10))
print("Strings")
pagos_str.head(10)

Numerical


,Número de Pago,Fechas de Pago,Día de Inicio de Cobertura,Mes de Pago,Año de Pago,Salario Diario Asignado,Relación Salario/UMA,Días cubiertos,Salario Asignado Mensual,% de pago respecto del salario asignado,Pago Mensual,Pago Acumulado,Salario Mínimo del año en curso,Salario Máximo del año en curso
1,1,2023-02-03,3,Febrero,2023,2593.5,25.0,26,67431.0,0.1117,7529.35,7529.35,207.44,2593.5
2,2,2023-03-01,1,Marzo,2023,2593.5,25.0,31,80398.5,0.1117,8977.30,16506.65,207.44,2593.5
3,3,2023-04-01,1,Abril,2023,2593.5,25.0,30,77805.0,0.1117,8687.71,25194.36,207.44,2593.5
4,4,2023-05-01,1,Mayo,2023,2593.5,25.0,31,80398.5,0.1117,8977.30,34171.66,207.44,2593.5
5,5,2023-06-01,1,Junio,2023,2593.5,25.0,30,77805.0,0.1117,8687.71,42859.37,207.44,2593.5
6,6,2023-07-01,1,Julio,2023,2593.5,25.0,31,80398.5,0.1117,8977.30,51836.67,207.44,2593.5
7,7,2023-08-01,1,Agosto,2023,2593.5,25.0,31,80398.5,0.1117,8977.30,60813.97,207.44,2593.5
8,8,2023-09-01,1,Septiembre,2023,2593.5,25.0,30,77805.0,0.1117,8687.71,69501.68,207.44,2593.5
9,9,2023-10-01,1,Octubre,2023,2593.5,25.0,31,80398.5,0.1117,8977.30,78478.98,207.44,2593.5
10,10,2023-11-01,1,Noviembre,2023,2593.5,25.0,30,77805.0,0.1117,8687.71,87166.69,207.44,2593.5


Strings


,Número de Pago,Fechas de Pago,Día de Inicio de Cobertura,Mes de Pago,Año de Pago,Salario Diario Asignado,Relación Salario/UMA,Días cubiertos,Salario Asignado Mensual,% de pago respecto del salario asignado,Pago Mensual,Pago Acumulado,Salario Mínimo del año en curso,Salario Máximo del año en curso
1,1,"Viernes, 3 de febrero de 2023",3,Febrero,2023,"$2,593.50",25.0,26,"$67,431.00",11.17%,"$7,529.35","$7,529.35",$207.44,"$2,593.50"
2,2,"Miércoles, 1 de marzo de 2023",1,Marzo,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$16,506.65",$207.44,"$2,593.50"
3,3,"Sábado, 1 de abril de 2023",1,Abril,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$25,194.36",$207.44,"$2,593.50"
4,4,"Lunes, 1 de mayo de 2023",1,Mayo,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$34,171.66",$207.44,"$2,593.50"
5,5,"Jueves, 1 de junio de 2023",1,Junio,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$42,859.37",$207.44,"$2,593.50"
6,6,"Sábado, 1 de julio de 2023",1,Julio,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$51,836.67",$207.44,"$2,593.50"
7,7,"Martes, 1 de agosto de 2023",1,Agosto,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$60,813.97",$207.44,"$2,593.50"
8,8,"Viernes, 1 de septiembre de 2023",1,Septiembre,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$69,501.68",$207.44,"$2,593.50"
9,9,"Domingo, 1 de octubre de 2023",1,Octubre,2023,"$2,593.50",25.0,31,"$80,398.50",11.17%,"$8,977.30","$78,478.98",$207.44,"$2,593.50"
10,10,"Miércoles, 1 de noviembre de 2023",1,Noviembre,2023,"$2,593.50",25.0,30,"$77,805.00",11.17%,"$8,687.71","$87,166.69",$207.44,"$2,593.50"


# Tabla Pivot de Pagos Mod. 40
### Parámetros de entrada

1. Tabla de Pagos numérica

In [18]:
def pivot_pagos40(df):
    pivot_table = pd.pivot_table(data=df,
                index = 'Mes de Pago',
                values = 'Pago Mensual',
                columns = 'Año de Pago',
                aggfunc = 'sum',
                fill_value=0,
                margins=True,
                margins_name='Total')
    # Rename margin row and column
    pivot_table.rename(index={pivot_table.index[-1]: 'Total por mes'},
                    columns={pivot_table.columns[-1]: 'Total por año'},
                    inplace=True)

    # Define the desired order
    order = ['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Total por mes']

    # Reindex the pivot table
    ordered_pivot_table = pivot_table.reindex(index=order)

    # Convert float values to currency strings
    ordered_pivot_table_string = ordered_pivot_table.map("${:,.2f}".format)

    total_payment = ordered_pivot_table['Total por año'].iloc[-1]
    total_payment = convert_double_currency(total_payment)
    return ordered_pivot_table, ordered_pivot_table_string, total_payment

risultati = pivot_pagos40(pagos_num)
pivot_table = risultati[0]
pivot_table_string = risultati[1]
display(pivot_table_string)
total_payment = risultati[2]
print(f"Total Payment: {total_payment}")

Año de Pago,2023,2024,2025,Total por año
Mes de Pago,,,,
Enero,$0.00,"$9,853.64","$10,730.79","$20,584.43"
Febrero,"$7,529.35","$9,217.92","$9,692.32","$26,439.59"
Marzo,"$8,977.30","$9,853.64",$0.00,"$18,830.94"
Abril,"$8,687.71","$9,535.78",$0.00,"$18,223.49"
Mayo,"$8,977.30","$9,853.64",$0.00,"$18,830.94"
Junio,"$8,687.71","$9,535.78",$0.00,"$18,223.49"
Julio,"$8,977.30","$9,853.64",$0.00,"$18,830.94"
Agosto,"$8,977.30","$9,853.64",$0.00,"$18,830.94"
Septiembre,"$8,687.71","$9,535.78",$0.00,"$18,223.49"


Total Payment: $232,903.62


## Crecimiento anual:

### Parámetros de entrada

1. Año de inicio
2. Valor de pensión
3. Año Final




In [19]:
def crecimiento_anual_pension_fcn(ano_pension, pension_final, year_max=2040):
    tabla = 'TABLAS/TABLA_INFLACION.txt'
    tabla = pd.read_csv(tabla, sep = '\t')
    year_max = min(2040, year_max)
    num_years = year_max - ano_pension
    year_array = np.arange(0, num_years + 1) + ano_pension
    idx_inicial = tabla.index[tabla['AÑO'] == ano_pension].tolist()[0]
    idx_final = tabla.index[tabla['AÑO'] == ano_pension + num_years].tolist()[0]

    inflacion_anual_numerico = tabla.iloc[idx_inicial:idx_final, 1]
    inflacion_anual_numerico = inflacion_anual_numerico.map(lambda x: float(x.replace('%','')))
    inflacion_anual_numerico = pd.concat([pd.Series([0]), inflacion_anual_numerico])

    inflacion_anual_acumulada = np.cumsum(inflacion_anual_numerico)
    pension_mensual = (inflacion_anual_acumulada/1e2 + 1) * pension_final

    tabla_crecimiento_num = pd.DataFrame({
        'Año': year_array.astype(int),
        'Inflación Anual': inflacion_anual_numerico,
        'Inflación Anual Acumulada' : inflacion_anual_acumulada,
        'Pensión recibida mensual' : pension_mensual
    }).round(2)

    tabla_crecimiento_str = tabla_crecimiento_num.copy()
    tabla_crecimiento_str['Inflación Anual'] = (tabla_crecimiento_str['Inflación Anual']).map("{:.3}%".format)
    tabla_crecimiento_str['Inflación Anual Acumulada'] = (tabla_crecimiento_str['Inflación Anual Acumulada']).map("{:.3}%".format)
    tabla_crecimiento_str['Pensión recibida mensual'] = [convert_double_currency(val) for val in tabla_crecimiento_str['Pensión recibida mensual']]


    return tabla_crecimiento_num, tabla_crecimiento_str

ano_inicio = 2025 #@param {type:"integer"}
valor_pension = 56110 #@param {type:"integer"}
year_max = 2040 #@param {type:"integer"}

tabla_crecimiento_num, tabla_crecimiento_str = crecimiento_anual_pension_fcn(ano_inicio, valor_pension, year_max)
display(tabla_crecimiento_str.head(10))
display(tabla_crecimiento_num.head(10))

,Año,Inflación Anual,Inflación Anual Acumulada,Pensión recibida mensual
0,2025,0.0%,0.0%,"$56,110.00"
22,2026,4.29%,4.29%,"$58,517.12"
23,2027,4.29%,8.58%,"$60,924.24"
24,2028,4.29%,12.9%,"$63,331.36"
25,2029,4.29%,17.2%,"$65,738.48"
26,2030,4.29%,21.4%,"$68,145.60"
27,2031,4.29%,25.7%,"$70,552.71"
28,2032,4.28%,30.0%,"$72,954.22"
29,2033,4.28%,34.3%,"$75,355.73"
30,2034,4.28%,38.6%,"$77,757.24"


,Año,Inflación Anual,Inflación Anual Acumulada,Pensión recibida mensual
0,2025,0.00,0.00,56110.00
22,2026,4.29,4.29,58517.12
23,2027,4.29,8.58,60924.24
24,2028,4.29,12.87,63331.36
25,2029,4.29,17.16,65738.48
26,2030,4.29,21.45,68145.60
27,2031,4.29,25.74,70552.71
28,2032,4.28,30.02,72954.22
29,2033,4.28,34.30,75355.73
30,2034,4.28,38.58,77757.24


# Tabla Pivot con Plotly

In [24]:
def heatmap_pagos40(pivot_table):
    # df should have 'Mes de Pago' as index, 'Año de Pago' as columns, and 'Pago Mensual' as values
    # Melt the DataFrame to long format
    df = pivot_table.copy()
    print(df.columns, df.index)
    df.drop('Total por mes', axis=0, inplace=True)
    df.drop('Total por año', axis=1, inplace=True)
    df_melted = df.reset_index().melt(id_vars=['Mes de Pago'], var_name='Año de Pago', value_name='Pago Mensual')
    # Create the heatmap
    fig = go.Figure(data=go.Heatmap(
        x=df_melted['Año de Pago'],
        y=df_melted['Mes de Pago'],
        z=df_melted['Pago Mensual'],
        colorscale='turbo',
        colorbar=dict(
            tickformat="$,.0f"
        ),
        text=df_melted.apply(lambda row: f'Mes: {row["Mes de Pago"]}<br>Año: {row["Año de Pago"]}<br>Pago Mensual: ${row["Pago Mensual"]:.2f}', axis=1),

        hoverinfo='text',
    ))

    fig.update_layout(
        title='Pagos Mensuales',
        xaxis=dict(
            nticks=36,
            title='Año de Pago',
            tickmode='linear',
            tick0=0,
            dtick=1,
            tickformat='d'
        ),
        xaxis_nticks=36,
        yaxis_nticks=12,
        xaxis_title='Año de Pago',
        yaxis_title='Mes de Pago',
    )

    # Add text annotations
    for i in range(len(df_melted)):
        fig.add_annotation(
            x=df_melted['Año de Pago'][i],
            y=df_melted['Mes de Pago'][i],
            text=f"${df_melted['Pago Mensual'][i]:,.0f}",
            showarrow=False,
            font=dict(size=16, color = 'white')
        )

    return fig.show()

heatmap_pagos40(pivot_table)

Index([2023, 2024, 2025, 'Total por año'], dtype='object', name='Año de Pago') Index(['Enero', 'Febrero', 'Marzo', 'Abril', 'Mayo', 'Junio', 'Julio',
       'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre',
       'Total por mes'],
      dtype='object', name='Mes de Pago')
